In [1]:
import numpy as np
import igraph
from transformers import pipeline
import colorsys
import matplotlib as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline


In [2]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# class roberta_stance_model:
#     def __init__(self):
#         self.tokenizer = AutoTokenizer.from_pretrained("roberta-large-mnli")
#         self.model = AutoModelForSequenceClassification.from_pretrained(
#             "roberta-large-mnli"
#         ).to("cuda")   # remove .to("cuda") if CPU only

#         # MNLI label mapping
#         self.label_dict = {
#             "ENTAILMENT": 1,
#             "CONTRADICTION": -1,
#             "NEUTRAL": 0
#         }

#         # HuggingFace id2label mapping
#         self.id2label = self.model.config.id2label
#     def forward(self, pairs, batch_size=32):
#         all_probs = []
#         all_labels = []

#         for i in range(0, len(pairs), batch_size):
#             batch = pairs[i:i+batch_size]

#             a_list = [a for a,b in batch]
#             b_list = [b for a,b in batch]

#             enc = self.tokenizer(
#                 a_list, b_list,
#                 padding=True,
#                 truncation=True,
#                 max_length=256,
#                 return_tensors="pt"
#             ).to(self.model.device)

#             with torch.no_grad():
#                 logits = self.model(**enc).logits
#                 probs = torch.softmax(logits, dim=-1)

#             pred_ids = logits.argmax(dim=-1).tolist()
#             labels = [self.label_dict[self.id2label[j]] for j in pred_ids]

#             all_probs.extend(probs.tolist())
#             all_labels.extend(labels)

#             # free memory
#             del enc, logits, probs

#         torch.cuda.empty_cache()
#         return all_probs, all_labels
from transformers import pipeline
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util


class roberta_stance_model:
    def __init__(self, sim_threshold=0.30, device=None):
        self.MODEL_NAME = "roberta-large-mnli"
        self.tokenizer = AutoTokenizer.from_pretrained(self.MODEL_NAME)
        self.model = AutoModelForSequenceClassification.from_pretrained(self.MODEL_NAME)
        self.model.eval()
        self.device = device if device else ("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)

        # HF NLI label mapping: 0=contradiction, 1=neutral, 2=entailment
        self.label_map = {0: -1, 1: 0, 2: 1}  # -1=contradiction, 0=neutral, 1=support
    @torch.no_grad()
    def forward(self, pairs: list[tuple[str, str]]):
        """
        pairs: list of (premise, hypothesis) tuples
        Returns: 
            output_probs: list of dicts {'ENTAILMENT':.., 'CONTRADICTION':.., 'NEUTRAL':..}
            output_label: list of stances (-1,0,1)
        """
        output_probs = []
        output_label = []

        # Tokenize all pairs in batch
        texts_A = [a for a, b in pairs]
        texts_B = [b for a, b in pairs]
        inputs = self.tokenizer(
            texts_A,
            texts_B,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=256
        ).to(self.device)

        outputs = self.model(**inputs)
        probs_batch = F.softmax(outputs.logits, dim=-1)  # shape: [batch, 3]

        for probs in probs_batch:
            # probs as dict with original keys
            prob_dict = {
                'ENTAILMENT': float(probs[2]),
                'CONTRADICTION': float(probs[0]),
                'NEUTRAL': float(probs[1])
            }
            output_probs.append(prob_dict)

            # pick stance by highest probability
            label_id = int(torch.argmax(probs))
            stance = {0: -1, 1: 0, 2: 1}[label_id]

            output_label.append(stance)

        return output_probs, output_label


In [3]:
embedder = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")


In [4]:
a = ''' The Open Question Argument argues that if moral properties were natural properties, then ""yes"" would logically follow from ""is [eg. pleasure] goodness?"" just like ""yes"" follows from ""is H20 water?"". However, intuitively the former question requires a much more open answer to the latter. This indicates that moral properties are not self-evidently natural.'''

b = '''It is possible for people to draw differing conclusions from the same facts because they have unique prior experiences and value judgments
 about the world that they will subconsciously rely on when evaluating the same evidence.'''
l = [a,a,a,a,a,a,a,a,a,a,a]

In [5]:
x = embedder.encode(l)
# print(F.cosine_similarity(torch.tensor(x),torch.tensor(y), dim = 0))

In [6]:
embedder.similarity(x, x[:5]//2)

tensor([[0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689],
        [0.5689, 0.5689, 0.5689, 0.5689, 0.5689]])

In [7]:
embedder.similarity(x[:5],1+x//2)

tensor([[0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410,
         0.5410, 0.5410],
        [0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410,
         0.5410, 0.5410],
        [0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410,
         0.5410, 0.5410],
        [0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410,
         0.5410, 0.5410],
        [0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410, 0.5410,
         0.5410, 0.5410]])

In [8]:
x.shape

(11, 384)

In [9]:
# MODEL_NAME = "roberta-large-mnli"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
# model.eval()

# label_map = {
#     0: "contradiction",
#     1: "neutral",
#     2: "entailment"  # treat as "supports"
# }
# import torch
# import torch.nn.functional as F

# @torch.no_grad()
# def classify_relation(premise, hypothesis):
#     """
#     Uses NLI model: premise -> hypothesis.
#     Returns one of: 'supports', 'contradicts', 'neutral'
#     """
#     inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True, max_length=256)
#     outputs = model(**inputs)
#     logits = outputs.logits
#     probs = F.softmax(logits, dim=-1)[0]
#     label_id = int(torch.argmax(probs))
#     nli_label = label_map[label_id]

#     if nli_label == "entailment":
#         return "supports", float(probs[label_id])
#     elif nli_label == "contradiction":
#         return "contradicts", float(probs[label_id])
#     else:
#         return "neutral", float(probs[label_id])

In [10]:
class Discourse():
    def __init__(self, stance_model):
        self.discourse_history = []
        self.stance_graph = igraph.Graph(directed = True)
        self.stance_model = stance_model
        self.no_stance_update_vertex_idx = 0
        self.no_edges = 0
        self.device =  "cuda" if torch.cuda.is_available() else "cpu" #declare device for emberdder
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2", device=self.device) #Check esentence embedding for pruning.
        # Define similarity matrix for pruning of neutral stance checks
        self.similarity_matrix = []
        self.similarity_threshold = 0.3
        self.prune_neutral = True

    def add_statements_vertices(self, usr_statements: list[tuple[int, str]]):
        no_statements = len(usr_statements)
        self.discourse_history += usr_statements
        start_vertex_idx = len(self.stance_graph.vs)
        end_vertex_idx = start_vertex_idx + no_statements
        new_vertex_id = range(start_vertex_idx, end_vertex_idx)
        new_vertex_membership = [user for user, statement in usr_statements] #Define membership for new vertices
        self.stance_graph.add_vertices(no_statements, attributes = {"v_id": new_vertex_id,
                                                                    "membership": new_vertex_membership })
        
    def update_stance_graph_edges(self):
        start_vertex_idx = self.no_stance_update_vertex_idx
        end_vertex_idx = len(self.discourse_history)
        
        if(start_vertex_idx == 0):
            # Here since we are initializing new == all
            input_new_to_all_edges = [(v1,v2) for v1 in range(start_vertex_idx, end_vertex_idx) for v2 in range(start_vertex_idx, end_vertex_idx) if v1!=v2] 
            output_new_to_all_edges , new_to_all_edge_stances, all_to_new_edge_stances = self.get_stance(input_new_to_all_edges)
            
            self.add_edges(output_new_to_all_edges, new_to_all_edge_stances)
        
        else:
            #edges from new vertex to all vertices #end_vertex_idx used in place of len(self.discourse_history)
            input_new_to_all_edges = [(new_vertex_idx, all_vertex_idx) for new_vertex_idx in range(start_vertex_idx, end_vertex_idx) for all_vertex_idx in range(0,end_vertex_idx) if new_vertex_idx!=all_vertex_idx]
        
            output_new_to_all_edges, new_to_all_edge_stances, all_to_new_edge_stances = self.get_stance(input_new_to_all_edges)
            
            new_to_all_edges = output_new_to_all_edges[:]
            all_to_new_edges = [(all_vertex_idx, new_vertex_idx) for new_vertex_idx, all_vertex_idx in new_to_all_edges]
            
            self.add_edges(new_to_all_edges, new_to_all_edge_stances)
            self.add_edges(all_to_new_edges, all_to_new_edge_stances)
        

        self.no_stance_update_vertex_idx = end_vertex_idx
        
    def add_edges(self, edges:list[tuple[int,int]], stances:list[int]):
        new_e_ids = range(self.no_edges, self.no_edges+len(edges))
        self.stance_graph.add_edges(edges, attributes = {"stance": stances, "e_id": new_e_ids})
        self.no_edges+=len(edges)
    
    def get_stance(self, edges: list[tuple[int,int]]):
        #change to the dict output expected by
        new_usr, new_statements = zip(*self.discourse_history[self.no_stance_update_vertex_idx:])
        if self.no_stance_update_vertex_idx == 0:
            old_usr, old_statements = [], []
        else:
            old_usr, old_statements = zip(*self.discourse_history[0: self.no_stance_update_vertex_idx])
        new_statements = list(new_statements)
        old_statements = list(old_statements)
        
        statements_pair = []
        reverse_statements_pair = []
        old_and_new_statements = old_statements + new_statements
        embeddings_new_statements = self.embedder.encode(new_statements) #(no. of new_statements,)
        embeddings_old_to_new_statemnts = self.embedder.encode(old_and_new_statements) #(total no. of statemetns)
        similarity_old_to_new_statements = self.embedder.similarity(embeddings_old_to_new_statemnts, embeddings_new_statements) #
        
        if(len(self.similarity_matrix) == 0):
            self.similarity_matrix = similarity_old_to_new_statements[:]
        else:
            for old_vertex_idx in range(len(self.similarity_matrix)):
                self.similarity_matrix[old_vertex_idx].extend(similarity_old_to_new_statements[old_vertex_idx])
            new_to_all_similarity = similarity_old_to_new_statements[len(self.similarity_matrix):]
            self.similarity_matrix.extend(new_to_all_similarity)
        
        neutral_pairs = []
        reverse_neutral_pairs = []
        
        new_edge_list = []
        for v1_id,v2_id in edges: 
            #here v_id is the same as idx from our setup where v_id is the same as vertex idx in discourse history 
            if(self.prune_neutral):
                if(self.similarity_matrix[v1_id][v2_id] < self.similarity_threshold):
                    neutral_pairs.append((v1_id, v2_id))
                    reverse_neutral_pairs.append((v2_id, v1_id))
                    continue
            
            statements_pair.append((self.discourse_history[v1_id][1], self.discourse_history[v2_id][1]))
            reverse_statements_pair.append((self.discourse_history[v2_id][1], self.discourse_history[v1_id][1]))
            new_edge_list.append((v1_id, v2_id))
        edge_probs, edge_stances = self.stance_model.forward(statements_pair + reverse_statements_pair)
        
        forward_edge_probs,  forward_edge_stances = edge_probs[:len(statements_pair)], edge_stances[:len(statements_pair)]
        reverse_edge_probs,  reverse_edge_stances = edge_probs[len(statements_pair):], edge_stances[len(statements_pair):]
        
        forward_neutral_edge_probs, forward_neutral_edge_stances = [0 for _ in range(len(neutral_pairs))], [0 for _ in range(len(neutral_pairs))] 
        reverse_neutral_edge_probs, reverse_neutral_edge_stances = [0 for _ in range(len(reverse_neutral_pairs))], [0 for _ in range(len(reverse_neutral_pairs))] 
        
        forward_edge_probs.extend(forward_neutral_edge_probs)
        forward_edge_stances.extend(forward_neutral_edge_stances)
        reverse_edge_probs.extend(reverse_neutral_edge_probs)
        reverse_edge_stances.extend(reverse_neutral_edge_stances)
        new_edge_list.extend(neutral_pairs)
        
        # reverse_edge_stances = edge_stances #Use to increase performace
        return new_edge_list, forward_edge_stances, reverse_edge_stances 
        
    
    def get_in_degree(self, allowed_stances: list[int], allowed_vertex_membership: list[int] = [], allowed_v_id: list[int] = [], allowed_e_id: list[int] = []):
        if(len(allowed_v_id) == 0): allowed_v_id = range(len(self.stance_graph.vs["v_id"]))
        if(len(allowed_e_id) == 0): allowed_e_id = range(len(self.stance_graph.es["e_id"]))
        if(len(allowed_vertex_membership) == 0): allowed_vertex_membership = range(len(self.stance_graph.es["v_id"]))

        membership_subgraph = self.stance_graph.subgraph(self.stance_graph.vs.select(lambda v: v['membership'] in allowed_vertex_membership and v.index in allowed_v_id))
        stance_subgraph = membership_subgraph.subgraph_edges(membership_subgraph.es.select(lambda e: e['stance'] in allowed_stances and e.index in allowed_e_id), delete_vertices = True)
        
        desired_subgraph = stance_subgraph
        
        
            
    def export_to_gephi(self, path="stance_graph.graphml"):
        """
        Export the entire stance graph to a GraphML file for Gephi.
        Preserves all node/edge attributes (v_id, membership, stance, e_id).
        """
        self.stance_graph.write_graphml(path)
        print(f"Exported to {path}")

In [11]:

class FallacyChecker():
    def __init__(self, stance_graph: igraph.Graph):
        self.stance_graph = stance_graph
    
    def get_cycle(self, allowed_vertex_membership: list[int], allowed_stances: list[int]) -> tuple[list[tuple], list[tuple]]:
        membership_subgraph = self.stance_graph.subgraph(self.stance_graph.vs.select(membership_in = allowed_vertex_membership))
        stance_subgraph = membership_subgraph.subgraph_edges(membership_subgraph.es.select(stance_in = allowed_stances), delete_vertices = True)

        basis_cycles_edge_idx = stance_subgraph.minimum_cycle_basis()
        basis_cycle_vertices_id = []
        basis_cycle_edges_id = []
        for edge_cycle in basis_cycles_edge_idx:
            temp_vertices = []
            temp_edge_id = []
            # print(edge_cycle)
            for edge in edge_cycle:
                # print(edge)
                v1_idx, v2_idx = stance_subgraph.get_edgelist()[edge]
                v1_id = stance_subgraph.vs["v_id"][v1_idx]
                v2_id = stance_subgraph.vs["v_id"][v2_idx]
                vertices_id = [v1_id, v2_id]
                edge_id = stance_subgraph.es["e_id"][edge]
                temp_edge_id.append(edge_id)                
                temp_vertices.extend(vertices_id)
                
            basis_cycle_edges_id.append(temp_edge_id)    
            basis_cycle_vertices_id.append(tuple(set(temp_vertices)))   
        # print("EID: ", basis_cycles_edge_idx["e_id"]) 
        return basis_cycle_edges_id, basis_cycle_vertices_id
    
    def get_contradiction(self, allowed_vertex_membership: list[int]):
        allowed_stances = [-1]
        
        membership_subgraph = self.stance_graph.subgraph(self.stance_graph.vs.select(membership_in = allowed_vertex_membership))
        contradiction_stance_subgraph = membership_subgraph.subgraph_edges(membership_subgraph.es.select(stance_in = allowed_stances), delete_vertices = True)
        
        contradiction_vertices_id =  []
        for edge in contradiction_stance_subgraph.get_edgelist():
            v1_idx, v2_idx = edge
            v1_id = contradiction_stance_subgraph.vs["v_id"][v1_idx]
            v2_id = contradiction_stance_subgraph.vs["v_id"][v2_idx]
            contradiction_vertices_id.append((v1_id, v2_id))
        return contradiction_vertices_id
    


In [12]:
def plot_stance_graph( stance_graph: igraph.Graph,
        allowed_stances: list[int],
        allowed_vertex_membership: list[int] = [],
        allowed_v_id: list[int] = [],
        allowed_e_id: list[int] = []
    ):
        g = stance_graph

        # ------------------------------
        # FAST DEFAULTS
        # ------------------------------
        if len(allowed_v_id) == 0:
            allowed_v_id = np.arange(g.vcount())

        if len(allowed_e_id) == 0:
            allowed_e_id = np.arange(g.ecount())

        if len(allowed_vertex_membership) == 0:
            allowed_vertex_membership = np.unique(g.vs["membership"])


        # ------------------------------
        # VERTEX FILTER (VECTORIZED)
        # ------------------------------
        v_memberships = np.array(g.vs["membership"])
        v_ids = np.arange(g.vcount())

        vertex_mask = np.isin(v_memberships, allowed_vertex_membership) & np.isin(v_ids, allowed_v_id)
        vertex_indices = np.where(vertex_mask)[0]

        sub_v = g.subgraph(vertex_indices)


        # ------------------------------
        # EDGE FILTER (VECTORIZED)
        # IMPORTANT: use stored e_id from original graph
        # ------------------------------
        sub_e_stance = np.array(sub_v.es["stance"])
        sub_e_id = np.array(sub_v.es["e_id"])      # from original graph

        edge_mask = np.isin(sub_e_stance, allowed_stances) & np.isin(sub_e_id, allowed_e_id)
        edge_indices = np.where(edge_mask)[0]

        final = sub_v.subgraph_edges(edge_indices, delete_vertices=True)


        # ------------------------------
        # UNIQUE COLOR PER MEMBERSHIP (FAST)
        # ------------------------------
        m = np.array(final.vs["membership"])
        uniq = np.unique(m)

        # evenly spaced HSV colors
        colors = np.array([colorsys.hsv_to_rgb(i/len(uniq), 0.75, 0.95) for i in range(len(uniq))], dtype=object)

        # vectorized remapping
        membership_to_color = {u: colors[i] for i, u in enumerate(uniq)}
        vcolor = [membership_to_color[x] for x in m]


        # ------------------------------
        # EDGE COLORS (NO LOOPS)
        # ------------------------------
        stance_map = {1: "green", 0: "gray", -1: "red"}
        ecolor = [stance_map[s] for s in final.es["stance"]]


        # ------------------------------
        # PLOT (FASTEST LAYOUT)
        # ------------------------------
        layout = final.layout("fr")   # can switch to "kk" if needed

        igraph.plot(
            final,
            layout=layout,
            vertex_color=vcolor,
            vertex_size=40,
            edge_color=ecolor,
            vertex_label=[str(v["v_id"]) for v in final.vs],
            bbox=(600, 600),
            margin=50
        )



In [13]:
from debate_datasets import Ethix_data
Ethix_dataset = Ethix_data(r"C:\All Codes\BTP\New_Igraph\Ethix_dataset.csv")

debate_idx = 1
scheme_list = Ethix_dataset.iloc[debate_idx]["Scheme"]
argument_text = Ethix_dataset.iloc[debate_idx]["Argument"]
print(scheme_list)
print(argument_text)
debate_data = list(zip(scheme_list, argument_text))
print(len(debate_data))
# Create list of tuples [(number, debate_text), ...]
# debate_data = [(num, debate) for num , debate in (scheme_list, debate_text)]


[1, 3, 1, 3, 0, 7, 3, 1, 4, 6, 0, 0, 4, 4, 2, 4, 4, 1, 1, 4, 0, 1, 7, 7, 1, 7, 2, 0, 5, 4, 2, 0]
['A one-size-fits-all prescription to good health is both ideologically and practically dangerous.', 'The UK government has made exceptions to the rule requiring masks for specific groups of people.\r\n This is an acknowledgement of the practical dangers of a one-size-fits-all prescription.', 'It is not reasonable for everyone to wear masks.', 'For people with certain underlying conditions and medical considerations, mask wearing is not possible.', 'Many rape survivors, for whom wearing a mask can trigger trauma and flashbacks, are being stigmatized for not wearing masks.', 'Mandating mask-wearing during a pandemic is like mandating safety/hand brakes on a car. It may never be required\r\n but is in everyone’s interest that it is there and working if it is required.', 'COVID-19 is universally transmitted through infected secretions and through respiratory droplets. ', 'During a world pandem

In [14]:
Ethix_dataset

,Debate,Argument,Scheme
0,Are moral properties natural properties?,[Moral anti-realism is correct and there are n...,"[4, 4, 3, 4, 3, 4, 0, 1, 3, 4, 4, 0, 7, 0, 0]"
1,Do people have a right to not wear a mask in p...,[A one-size-fits-all prescription to good heal...,"[1, 3, 1, 3, 0, 7, 3, 1, 4, 6, 0, 0, 4, 4, 2, ..."
2,Do we have a moral duty to intervene in nature...,[Acting out of empathy to help others is in hu...,"[1, 7, 1, 5, 6, 5, 3, 0, 2, 2, 5, 5, 0, 3, 3, ..."
3,Free Will or Determinism: Do we have free will?,[Retrocausality is possible in some interpreta...,"[0, 4, 3, 3, 0, 3, 3, 4, 4, 3, 4, 4, 0, 0, 7, ..."
4,Has social media been good for humanity?,"[Humans can collaborate, meet, and communicate...","[2, 2, 3, 5, 4, 5, 0, 4, 0, 3, 0, 0, 5, 0, 5, ..."
5,Is cannibalism ethically permissible?,[The survivors of Uruguayan Air Force Flight 5...,"[0, 0, 1, 7, 1, 1, 1, 1, 1, 1, 4, 5, 1, 0, 0, ..."
6,Is capitalism the most moral system?,[While the United States has been relatively p...,"[0, 2, 2, 1, 5, 6, 4, 1, 1, 1, 7, 5, 5, 0, 1, ..."
7,Is cloning humans ethical?,[Individuals should have the autonomy to make ...,"[2, 7, 7, 0, 5, 4, 5, 0, 0, 6, 2, 0, 1, 1, 3, ..."
8,Is it ethically wrong to watch pornography?,[Plenty of porn is made in an ethical and safe...,"[1, 1, 1, 1, 5, 3, 7, 3, 0, 2, 1, 5, 0, 3, 7, ..."
9,Is it ok to incentivise moral behavior?,[Incentivizing moral behavior is de facto an o...,"[2, 5, 5, 1, 3, 7, 7, 2, 0, 3, 0, 4, 0, 1, 7, ..."


In [15]:
usr = 4
dp = {}
for k,v in debate_data:
    dp.setdefault(k, []).append(v)
for u in sorted(dp.keys()):
    print(u)
    print(dp[u])


0
['Many rape survivors, for whom wearing a mask can trigger trauma and flashbacks, are being stigmatized for not wearing masks.', "Mandatory face mask wearing, and the undercutting of people's rights not to wear one, provoked protests in Spain.", 'In many places, such as Florida, mask wearing is not mandatory, and yet there have been protests \r\n(https://abc7news.com/viral-target-anti-mask-video-protesters-parade-around-store-demand-customers-remove-face-masks-rant-caught-on/6428893/).', 'This right to bodily autonomy is not without limits. Prostitution is illegal in many countries.', 'Public health has long been used as a justification to curtail civil liberties, such as mandatory quarantine for those with yellow fever in 1799.', 'Even the WHO advice on masks has changed over time.']
1
['A one-size-fits-all prescription to good health is both ideologically and practically dangerous.', 'It is not reasonable for everyone to wear masks.', 'During a world pandemic, it is important to be

In [16]:
# debate_data = [[0,"Climate change is primarily caused by human activities like burning fossil fuels."],
# [0, "Some scientists argue that climate change is part of a natural cycle of the Earth."],
# [0,"If climate change were purely natural, CO2 levels would not have risen so sharply after industrialization."],
# [0, "Therefore, human activities are the main driver of recent climate change."],
# [0, "But if human activities are the main driver, then stopping emissions should immediately stop all climate change, which is not true."],
# [0, "So human activities cannot be the main driver of climate change."]]
# model = roberta_stance_model("dict")




In [17]:

# model = roberta_stance_model("list")
# d = Discourse(model)

# d.add_statements_vertices(debate_data)
# d.update_stance_graph_edges()
# g = d.stance_graph
# allowed_stances = [1, -1]  # Include all stances you want to display

# # Filter graph by allowed stances
# g = g.subgraph_edges(g.es.select(stance_in=allowed_stances), delete_vertices=True)

# # Generate unique colors for each membership
# unique_memberships = sorted(set(g.vs['membership']))
# n = len(unique_memberships)

# membership_to_color = {}
# for i, membership in enumerate(unique_memberships):
#     hue = i / n
#     r, gr, b = colorsys.hsv_to_rgb(hue, 0.8, 0.9)
#     membership_to_color[membership] = (r, gr, b)

# # Assign vertex colors based on membership
# vcolor = [membership_to_color[m] for m in g.vs['membership']]

# # Assign edge colors based on stance
# stance_color_map = {1: 'green', 0: 'gray', -1: 'red'}
# ecolor = [stance_color_map[s] for s in g.es['stance']]

# # Layout
# layout = g.layout('fr')

# # Save plot
# igraph.plot(
#     g,
#     layout=layout,
#     vertex_color=vcolor,
#     vertex_size=40,
#     edge_color=ecolor,
#     vertex_label=[str(v['v_id']) for v in g.vs],
#     bbox=(600, 600),
#     margin=50
# )

In [18]:
# print(d.stance_graph.vs["v_id"])

In [19]:
# d.export_to_gephi()

In [20]:
import os
import igraph as ig
import colorsys
import time

Ethix_dataset = Ethix_data(r"C:\All Codes\BTP\New_Igraph\Ethix_dataset.csv")

def graph_Ethix_dataset():
    # Create folder for images
    output_folder = "graph_images"
    os.makedirs(output_folder, exist_ok=True)

    # Create or open log file
    log_file = os.path.join(output_folder, "graph_benchmark.txt")
    with open(log_file, "w") as f_log:
        f_log.write("DebateIdx\tVertexCount\tEdgeCount\tTimeTaken(s)\n")  # header

    for debate_idx in range(len(Ethix_dataset)):
        start_time = time.time()

        model = roberta_stance_model("dict")
        d = Discourse(model)

        scheme_list = Ethix_dataset.iloc[debate_idx]["Scheme"]
        argument_text = Ethix_dataset.iloc[debate_idx]["Argument"]
        debate_data = list(zip(scheme_list, argument_text))

        d.add_statements_vertices(debate_data)
        d.update_stance_graph_edges()

        g = d.stance_graph
        allowed_stances = [1, -1]
        vertex_count = len(g.vs)
        edge_count = len(g.es)
        stances = g.es["stance"]

        neutral_edges = stances.count(0)
        support_edges = stances.count(1)
        contradict_edges = stances.count(-1)


        # Filter graph by allowed stances
        g = g.subgraph_edges(g.es.select(stance_in=allowed_stances), delete_vertices=True)

        # Vertex colors
        unique_memberships = sorted(set(g.vs['membership']))
        n = len(unique_memberships)
        membership_to_color = {}
        for i, membership in enumerate(unique_memberships):
            hue = i / n
            r, gr, b = colorsys.hsv_to_rgb(hue, 0.8, 0.9)
            membership_to_color[membership] = (r, gr, b)
        vcolor = [membership_to_color[m] for m in g.vs['membership']]

        # Edge colors
        stance_color_map = {1: 'green', 0: 'gray', -1: 'red'}
        ecolor = [stance_color_map[s] for s in g.es['stance']]

        # Layout and save
        layout = g.layout('fr')
        output_path = os.path.join(output_folder, f"graph_{debate_idx}.png")
        ig.plot(
            g,
            target=output_path,
            layout=layout,
            vertex_color=vcolor,
            vertex_size=40,
            edge_color=ecolor,
            vertex_label=[str(v['v_id']) for v in g.vs],
            bbox=(600, 600),
            margin=50
        )

        end_time = time.time()
        time_taken = end_time - start_time

        # Log to file
        with open(log_file, "a") as f_log:
            f_log.write(f"{debate_idx}\t{vertex_count}\t{edge_count}\t{time_taken:.2f}\n")

        print(
            f"Graph {debate_idx} saved | "
            f"Vertices: {vertex_count}, "
            f"Edges: {edge_count}, "
            f"Neutral edges: {neutral_edges}, "
            f"Support edges: {support_edges}, "
            f"Contradiction edges: {contradict_edges}, "
            f"Time taken: {time_taken:.2f}s"
        )



In [21]:
graph_Ethix_dataset()


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Graph 0 saved | Vertices: 15, Edges: 210, Neutral edges: 186, Support edges: 4, Contradiction edges: 20, Time taken: 25.46s


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Graph 1 saved | Vertices: 32, Edges: 992, Neutral edges: 923, Support edges: 9, Contradiction edges: 60, Time taken: 565.20s


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Graph 3 saved | Vertices: 17, Edges: 272, Neutral edges: 253, Support edges: 1, Contradiction edges: 18, Time taken: 51.96s


Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


: 

: 

In [ ]:
print(d.stance_model.forward([(argument_text[13], argument_text[2])]))
print(d.stance_model.forward([(argument_text[2], argument_text[13])]))

In [ ]:
import igraph as ig

g = d.stance_graph
allowed_stances = [1]
# Assign vertex memberships (for colors)
g = g.subgraph_edges(g.es.select(stance_in = allowed_stances), delete_vertices = True)
# Map memberships to vertex colors - more distinct colors
vertex_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']  # modern color palette
vcolor = [vertex_colors[m % len(vertex_colors)] for m in g.vs['membership']]

# Map stance to edge colors
stance_color_map = {1: '#2E8B57', 0: '#808080', -1: '#DC143C'}  # richer colors
ecolor = [stance_color_map[s] for s in g.es['stance']]

# Layout
layout = g.layout('fr')  # Fruchterman-Reingold layout

# Enhanced plotting with thinner, neater edges
visual_style = {
    'layout': layout,
    'vertex_color': vcolor,
    'vertex_size': 40,
    'vertex_frame_width': 1.5,
    'vertex_frame_color': 'white',
    'vertex_label': [f"V{i}" for i in range(g.vcount())],
    'vertex_label_color': 'black',
    'vertex_label_size': 12,
    'vertex_label_dist': 1.2,
    'edge_color': ecolor,
    'edge_width': 1.5,  # Thinner edges
    'edge_arrow_size': 0.8,  # Smaller arrow heads
    'edge_arrow_width': 0.8,
    'edge_curved': 0.1,  # Less curvature for straighter edges
    'bbox': (800, 800),
    'margin': 80,
    'background': '#f5f5f5'
}

# Plot the graph
ig.plot(g, **visual_style)

In [ ]:
import igraph as ig

g = d.stance_graph
allowed_stances = [1]
# Assign vertex memberships (for colors)
g = g.subgraph_edges(g.es.select(stance_in = allowed_stances), delete_vertices = True)

# Map memberships to vertex colors - more distinct colors
vertex_colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']  # modern color palette
vcolor = [vertex_colors[m % len(vertex_colors)] for m in g.vs['membership']]

# Map stance to edge colors
stance_color_map = {1: '#2E8B57', 0: '#808080', -1: '#DC143C'}  # richer colors
ecolor = [stance_color_map[s] for s in g.es['stance']]

# Layout
layout = g.layout('fr')  # Fruchterman-Reingold layout

# Enhanced plotting with thinner, neater edges
visual_style = {
    'layout': layout,
    'vertex_color': vcolor,
    'vertex_size': 40,
    'vertex_frame_width': 1.5,
    'vertex_frame_color': 'white',
    'vertex_label': [f"V{i}" for i in range(g.vcount())],
    'vertex_label_color': 'black',
    'vertex_label_size': 12,
    'vertex_label_dist': 1.2,
    'edge_color': ecolor,
    'edge_width': 1.5,  # Thinner edges
    'edge_arrow_size': 0.8,  # Smaller arrow heads
    'edge_arrow_width': 0.8,
    'edge_curved': 0.1,  # Less curvature for straighter edges
    'bbox': (800, 800),
    'margin': 80,
    'background': '#f5f5f5'
}

# Plot the graph
ig.plot(g, **visual_style)